link to the notebook: https://www.kaggle.com/samuelnordmann/titanic-binary-classification

In this notebook I present a studycase of binary classification on the Titanic dataset. The task consists in predicting what passenger survived in the Titanic crash.

This dataset is classical and well studied. Here I propose my own original solution achieving 77.8% accuracy on the test set, and thus ranks in the top 25% at time of submission.

Most of the work lies in the data engineering that I detail and try to explain as clearly as possible. Then, I train a XGB Classifier on 7 features selected by Mutual Information score.

In [151]:
import numpy as np 
import pandas as pd 
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Load, explore the data, and feature engineering

## Load

We first load the data and print the first rows.

In [152]:
#Load the dataset
data=pd.read_csv("../input/titanic/train.csv",index_col="PassengerId")
y=data["Survived"]
X=data.drop(labels=["Survived"], axis=1)

Let us take a glance at the input data

In [153]:
print('number of data inputs = ',len(X.index))
X.head()

and the first rows of the labels

In [154]:
y.head()

Here, 1 means that the passenger survived ; 0 means that the passenger died

Let us check the types and number of missing values

In [155]:
X.info()

Let us explore and engineer each feature one by one

## Feature engineering

### Pclass

The feature Pclass only takes the values 1,2,3:

In [156]:
X.Pclass.value_counts()

Since this feature represents the class category of the passenger's ticket, it is a categorical feature (even though the type is numerical). We decide to encode this feature as one-hot vectors.

In [157]:
oh= pd.get_dummies(X.Pclass,prefix="Pclass")
X = X.drop(labels='Pclass',axis=1).join(oh)

### Name

The only relevant information about name of the passenger should be whether the person has a title such as 'Mr.', or 'Mrs.', etc.

In [158]:
X['Title'] = X.Name.str.extract(' ([A-Za-z]+)\.', expand = False)
pd.crosstab(X.Title, X.Sex)

We decide to replace the rare titles by the string 'Rare', to gather the titles 'Mlle' and 'Ms', to gather the titles 'Mme' and 'Mrs'.

In [159]:
rare_titles = ['Capt', 'Col', 'Countess', 'Don', 'Dona', 'Dr', 'Jonkheer', 'Lady',
           'Major', 'Rev', 'Sir']

X.Title.replace(rare_titles, 'Rare', inplace = True)
X.Title.replace(['Mlle', 'Ms'], 'Miss', inplace = True)
X.Title.replace(['Mme'], 'Mrs', inplace = True)
pd.crosstab(X.Title, X.Sex)

In [160]:
sns.countplot(x=X.Title,hue=data.Survived)

In [161]:
oh= pd.get_dummies(X.Title,prefix="Title")
X = X.drop(labels=['Name','Title'],axis=1).join(oh)
#X.Name = (X.Name.str.contains('Mr.')|X.Name.str.contains('Mrs.')).astype(np.uint8)

### Sex

The column 'Sex' takes only two values:

In [162]:
X.Sex.value_counts()

We encode male as 0 and female as 1:X.Sex= (X.Sex=='female')

In [163]:
X.Sex= (X.Sex=='female').astype(np.uint8)

In [164]:
sns.countplot(data=data,x='Sex',hue="Survived")

### Age

In [165]:
sns.kdeplot(X.Age,hue=y, shade=True)

Let us check the ratio of missing value for this feature.

In [166]:
ratio = np.round(X.Age.isna().sum()/len(X.index)*100, decimals=1)
print('The percentage of missing ages is ', ratio,'%')

This ratio is quite high and so dropping the raws with missing values seems link a bad idea. We proceed to a simple mean imput, i.e., we replace the missing values by the average Age of the whole dataset.

In [167]:
X.Age[X.Age.isna()] = X.Age.mean()

Let us now look at the stastics of the feature

In [168]:
X.Age.describe()

We decide to rescale the feature by substracting the mean and dividing by the standard deviation:

In [169]:
X.Age = (X.Age-X.Age.mean())/X.Age.std()

### SibSp

In [170]:
sns.countplot(data=data,x='SibSp',hue="Survived")

In [171]:
X.SibSp.value_counts()

SibSp takes 7 different values, which is a slightly high for a one-hot encoding to be reasonable. Moreover, most of the dataset takes the values 0 and 1, while the values above 1 only represent a small percentage of the dataset:

In [172]:
ratio = np.round((X.SibSp>1).sum()/len(X.index)*100,decimals=1)
print("Percentage of the dataset whose SibSp is strictly above one = ",ratio,"%")

Since SibSp seems like an ordinal feature, we decide to gather in one categorie all the values strictly above 1 and to give them the value 2. 

In [173]:
X.SibSp.clip(upper=2,inplace=True)

We then proceed to a one-hot encoding:

In [174]:
oh= pd.get_dummies(X.SibSp,prefix="SibSp")
X = X.drop(labels='SibSp',axis=1).join(oh)

### Parch

In [175]:
sns.countplot(data=data,x='Parch',hue="Survived")

In [176]:
X.Parch.value_counts()

We treat the Parch feature similarily to SibSp, by clipiing the value to 2 and then do a one-hot encoding.

In [177]:
X.Parch.clip(upper=2,inplace=True)
oh= pd.get_dummies(X.Parch,prefix="Parch")
X = X.drop(labels='Parch',axis=1).join(oh)

### Ticket

We simply drop this feature:

In [178]:
X.drop(labels='Ticket',axis=1,inplace=True)

### Fare

In [179]:
sns.kdeplot(data=data,x='Fare', shade=True,hue="Survived")

In [180]:
X.Fare.describe()

We decide to rescale the feature by substracting the mean and dividing by the standard deviation:

In [181]:
X.Fare = (X.Fare-X.Fare.mean())/X.Fare.std()

### Cabin

Here we have to tackle two issues: missing value and extracting valuable information from the Cabin number.

First, let us replace the missing value by the token 'U'

In [182]:
X.Cabin[X.Cabin.isna()]='U'

Then, it seems reasonable to only keep the prefix letter of the cabin number (which could contain information on the category or location of the cabin).

In [183]:
X.Cabin=X.Cabin.str[0] #select the first letter
X.Cabin.value_counts() #print the statistics

In [184]:
sns.countplot(data=data,x=X.Cabin,hue="Survived")

Let us perform a Chi2 test for independence for each value of Cabin taken separately.

In [185]:
from scipy.stats import chi2_contingency
for letter in X.Cabin.unique():
    cab_binary = X.Cabin==letter
    contigency = pd.crosstab(cab_binary, y) 
    c, p, dof, expected = chi2_contingency(contigency)
    p = np.round(p*100,decimals=3)
    print('Chi2 test for the independence of Cabin = ',letter,'. p-value =',p,"%")
    

We see from the statistics and the plots that most of the valuable information on the cabin letter is whether this letter is "U" (corresponding to missing data).

Let us one-encode this feature while discarding the categories G, A, F, T which have a poor p-value.

In [186]:
oh = pd.get_dummies(X.Cabin,prefix="Cabin")
categories_to_drop = ['Cabin_'+i for i in ['G','A','F','T']]
oh = oh.drop(labels = categories_to_drop, axis = 1) 

X = X.drop(labels='Cabin',axis=1).join(oh)

### Embarked

In [187]:
sns.countplot(data=data,x='Embarked',hue="Survived")

In [188]:
X.Embarked.value_counts()

We proceed to a one-hot encoding

In [189]:
oh= pd.get_dummies(X.Embarked,prefix="Embarked")
X = X.drop(labels='Embarked',axis=1).join(oh)

## Conclusion of feature engineering 

In [190]:
X.info()

In [191]:
X.corr()

## Feature scoring

We rank the features using the Mutal Information Score.

In [192]:
from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y):
    X = X.copy()
    discrete_features = [X.columns.get_loc(column_name) for column_name in X.dtypes[X.dtypes=='uint8'].index] 
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

    
  
mi_scores = make_mi_scores(X, y)
X = X[mi_scores.index]  
sns.barplot(x=mi_scores, y=mi_scores.index)


We also perform a simple logistic regression an plot the results

In [193]:
from sklearn.linear_model import LogisticRegression

model_LogReg = LogisticRegression(solver='liblinear')
model_LogReg.fit(X, y)

coefs=pd.DataFrame({'coefficient':model_LogReg.coef_[0]}, index = X.columns)

coefs.sort_values('coefficient', key=abs, ascending=False,inplace=True)


sns.barplot(y=coefs.index, x=coefs['coefficient'])

print('intercept = ',model_LogReg.intercept_[0])


## Splitting

In [194]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val= train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=1,shuffle=True)

# Models

In [195]:
from sklearn.metrics import accuracy_score
import random

## XGB Classifier

### Hyperparameter search

In [196]:
scores_XGB=pd.DataFrame(columns = ['k','n_estimators','max_depth','score'])

In [197]:
from xgboost import XGBClassifier

if True: #To conveniently choose to not execute this cell at test time
    for _ in range(300): #We iterate to search best hyperparameters

        #Hyperparameters
        k= random.randint(6, 10) #number of selected features
        n_estimators = random.randint(50, 300) #Number of boosting rounds
        max_depth = random.randint(5, 7) #Maximum tree depth for base learners.

        model_XGBClassifier = XGBClassifier(n_estimators=n_estimators,max_depth=max_depth,use_label_encoder=False,eval_metric="error",random_state=0)

        selected_features = mi_scores.index[:k] #Selected features
        model_XGBClassifier.fit(X_train[selected_features], y_train, 
                            eval_set=[(X_val[selected_features],y_val)],
                           eval_metric='error',
                           verbose=False)

        preds = model_XGBClassifier.predict(X_val[selected_features])
        score = accuracy_score(y_val,preds)
        max_depth = model_XGBClassifier.get_params()['max_depth']
        score = pd.Series({'k':k,'n_estimators':n_estimators,'max_depth':max_depth,'score':score})
        scores_XGB = scores_XGB.append(score,ignore_index=True)

    scores_XGB = scores_XGB.sort_values(["score"],ascending=False)
    scores_XGB



In [198]:
misclassified=X_val.loc[y_val[y_val!=preds].index]
misclassified.describe()

In [199]:
sns.scatterplot(x=scores_XGB.k,y=scores_XGB.score,hue=scores_XGB.n_estimators)

k=6 seems like a good choice.

In [200]:
scores_XGB_k7= scores_XGB[scores_XGB.k.isin([5,6,7,8])]

sns.scatterplot(x=scores_XGB_k7.max_depth,y=scores_XGB_k7.score,hue=scores_XGB.k)

max_depth = 7 is a good choice

In [201]:
scores_XGB_k7= scores_XGB[scores_XGB.k.isin([6,7,8])]

sns.scatterplot(x=scores_XGB_k7.n_estimators,y=scores_XGB_k7.score,hue=scores_XGB.k)

n_estimators = 200 seems like a good choice

# Test

## Preprocessing of the test set

In [203]:
X_test=pd.read_csv("../input/titanic/test.csv",index_col="PassengerId")
X_test.info()

In [204]:
#Pclass
oh= pd.get_dummies(X_test.Pclass,prefix="Pclass")
X_test = X_test.drop(labels='Pclass',axis=1).join(oh)

#Name
X_test['Title'] = X_test.Name.str.extract(' ([A-Za-z]+)\.', expand = False)
X_test.Title.replace(rare_titles, 'Rare', inplace = True)
X_test.Title.replace(['Mlle', 'Ms'], 'Miss', inplace = True)
X_test.Title.replace(['Mme'], 'Mrs', inplace = True)

oh= pd.get_dummies(X_test.Title,prefix="Title")
X_test = X_test.drop(labels=['Name','Title'],axis=1).join(oh)

#Sex
X_test.Sex= (X_test.Sex=='female').astype(np.uint8)

#Age
X_test.Age[X_test.Age.isna()] = X.Age.mean()
X_test.Age = (X_test.Age-X.Age.mean())/X.Age.std()

#SibSp
X_test.SibSp.clip(upper=2,inplace=True)
oh= pd.get_dummies(X_test.SibSp,prefix="SibSp")
X_test = X_test.drop(labels='SibSp',axis=1).join(oh)

#Parch
X_test.Parch.clip(upper=2,inplace=True)
oh= pd.get_dummies(X_test.Parch,prefix="Parch")
X_test = X_test.drop(labels='Parch',axis=1).join(oh)

#Ticket
X_test.drop(labels='Ticket',axis=1,inplace=True)

#Fare
X_test.Fare[X_test.Fare.isna()] = X.Fare.mean()
X_test.Fare = (X_test.Fare-X.Fare.mean())/X.Fare.std()


#Cabin
X_test.Cabin[X_test.Cabin.isna()]='U'
X_test.Cabin=X_test.Cabin.str[0] #select the first letter
oh = pd.get_dummies(X_test.Cabin,prefix="Cabin")
categories_to_drop = ['Cabin_'+i for i in ['G','A','F','T']]
for category in categories_to_drop:
    try:
        oh = oh.drop(labels = category, axis = 1)
    except:
        pass
       
X_test = X_test.drop(labels='Cabin',axis=1).join(oh)


#Embarked
oh= pd.get_dummies(X_test.Embarked,prefix="Embarked")
X_test = X_test.drop(labels='Embarked',axis=1).join(oh)

X_test.info()

### Final XGBClassifier model

In [205]:
k= 6 #number of selected features
n_estimators = 200 #Number of boosting rounds
max_depth = 7 #Maximum tree depth for base learners.

model_XGBClassifier = XGBClassifier(n_estimators=n_estimators,
                                    max_depth=max_depth,
                                    use_label_encoder=False,
                                    eval_metric="error",
                                    random_state=0)

selected_features = mi_scores.index[:k] #Selected features
model_XGBClassifier.fit(X[selected_features], y, 
                   eval_metric='error',
                   verbose=False)

preds = model_XGBClassifier.predict(X_test[selected_features])


In [206]:
submission = pd.DataFrame({'Survived':preds}, index=X_test.index)
submission.to_csv("submission.csv")